## Nume studenti:
- Alexandra Manole
- Teodor Mihaescu

## Grupa: 382

# Data Set 2: Energy Efficency
### (Missing values: no)

In [1]:
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV, RandomizedSearchCV

In [2]:
# Citeste datele
data = pd.read_csv('./Datasets/energy.csv').iloc[:768,:-2]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    float64
 6   X7      768 non-null    float64
 7   X8      768 non-null    float64
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(10)
memory usage: 60.1 KB


In [3]:
# Pregateste datele
X = data.loc[:, :'X8']
# y = data.loc[:, 'Y1':]
y1 = data.loc[:, 'Y1']
y2 = data.loc[:, 'Y2']

In [4]:
# Transformam problema dintr-una de regresie intr-una de clasificare, dand intervale de clasificare y-ilor
"""
Observam ca y1 si y2 se incadreaza intre urmatoarele valori:
y1: 6.01-43.1
y2: 10.9-48.03

Vom imparti datele in urmatoarele clase:
y1: 6-10, 10-20, 20-30, 30-40, 40-50
y2: 10-20, 20-30, 30-40, 40-50
"""
y1 = np.where(np.logical_and(y1>=6, y1<10), 0, y1)
y1 = np.where(np.logical_and(y1>=10, y1<20), 1, y1)
y1 = np.where(np.logical_and(y1>=20, y1<30), 2, y1)
y1 = np.where(np.logical_and(y1>=30, y1<40), 3, y1)
y1 = np.where(np.logical_and(y1>=40, y1<50), 4, y1)

y2 = np.where(np.logical_and(y2>=10, y2<20), 0, y2)
y2 = np.where(np.logical_and(y2>=20, y2<30), 1, y2)
y2 = np.where(np.logical_and(y2>=30, y2<40), 2, y2)
y2 = np.where(np.logical_and(y2>=40, y2<50), 3, y2)

y = np.stack((y1, y2), axis=1)

In [5]:
# Scaleaza datele
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
# y = min_max_scaler.fit_transform(y)

In [6]:
# Separarea setului de date
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, shuffle=True)

In [7]:
# Verifica daca sunt egal distribuite
print(Counter(y_train[:,0]))
print(Counter(y_test[:,0]))
print("\n")
print(Counter(y_train[:,1]))
print(Counter(y_test[:,1]))

Counter({1.0: 247, 3.0: 114, 2.0: 109, 4.0: 28, 0.0: 14})
Counter({1.0: 130, 2.0: 60, 3.0: 53, 4.0: 7, 0.0: 6})


Counter({0.0: 231, 2.0: 149, 1.0: 105, 3.0: 27})
Counter({0.0: 117, 2.0: 67, 1.0: 60, 3.0: 12})


## Model 1: LogisticRegression

In [8]:
# Model
"""
"""
model1 = MultiOutputClassifier(LogisticRegression(random_state=0, multi_class='multinomial'))
model1.fit(X_train, y_train)
y_hat = model1.predict(X_test)

In [9]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[2. 1.]
 [1. 0.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[2. 1.]
 [1. 0.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [0. 0.]
 [2. 1.]
 [2. 2.]
 [1. 0.]
 [1. 0.]]

Failed:	[37 61]


In [10]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [11]:
# Custom scoreers for multiclass output
f1_scorer = metrics.make_scorer(metrics.f1_score, greater_is_better=False, average='micro')
accuracy_scorer = metrics.make_scorer(metrics.accuracy_score, greater_is_better=True)

In [12]:
# Cross Validation Y1
results_train = cross_validate(model1, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099802, 0.00099707, 0.00099683, 0.00200248, 0.        ]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_f1_scorer': array([nan, nan, nan, nan, nan]),
 'train_f1_scorer': array([nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan])}

In [13]:
# Cross Validation Y2
results_train = cross_validate(model1, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099874, 0.00097704, 0.        , 0.0009985 , 0.        ]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_f1_scorer': array([nan, nan, nan, nan, nan]),
 'train_f1_scorer': array([nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan])}

In [14]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {
    'C' : np.linspace(1000, 2000, 10),# C = 1/lambda ; C mare => lambda mic
    'solver': ['saga', 'lbfgs']}
grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [15]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.8868024132730016
{'C': 1888.888888888889, 'solver': 'lbfgs'}


In [16]:
# Cautare hiperparametrii optimi -RandomSearch
randomized_search = RandomizedSearchCV(estimator = LogisticRegression(), param_distributions=parameter_grid, random_state=0,  scoring='accuracy', cv=4 )
randomized_search.fit(X_train, y1_train)
results_rscv = cross_val_score(randomized_search, X_test, y1_test, cv=5)

In [17]:
print(results_rscv.mean())
print(randomized_search.best_params_)

0.8868024132730016
{'solver': 'lbfgs', 'C': 1888.888888888889}


## Model 2: KNeighborsClassifier

In [18]:
# Model
"""
"""
model2 = KNeighborsClassifier()
model2.fit(X_train, y_train)
y_hat = model2.predict(X_test)

In [19]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[1. 1.]
 [1. 1.]
 [1. 0.]
 [2. 2.]
 [3. 2.]
 [0. 0.]
 [2. 1.]
 [2. 2.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[2. 1.]
 [1. 0.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [0. 0.]
 [2. 1.]
 [2. 2.]
 [1. 0.]
 [1. 0.]]

Failed:	[36 54]


In [20]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [21]:
# Cross Validation Y1
results_train = cross_validate(model2, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00299072, 0.00199771, 0.00099492, 0.00199413, 0.0019486 ]),
 'score_time': array([0.00997472, 0.00996876, 0.01096797, 0.01196742, 0.00900555]),
 'test_f1_scorer': array([-0.68181818, -0.78571429, -0.76623377, -0.67320261, -0.74509804]),
 'train_f1_scorer': array([-0.97882736, -0.96091205, -0.96091205, -0.95121951, -0.96910569]),
 'test_accuracy': array([0.68181818, 0.78571429, 0.76623377, 0.67320261, 0.74509804]),
 'train_accuracy': array([0.97882736, 0.96091205, 0.96091205, 0.95121951, 0.96910569])}

In [22]:
# Cross Validation Y2
results_train = cross_validate(model2, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.0019927 , 0.00199294, 0.00199556, 0.00195217, 0.00203514]),
 'score_time': array([0.01595879, 0.01092839, 0.009974  , 0.00897598, 0.00793362]),
 'test_f1_scorer': array([-0.74675325, -0.67532468, -0.64935065, -0.60784314, -0.7124183 ]),
 'train_f1_scorer': array([-0.88599349, -0.88436482, -0.89576547, -0.88780488, -0.89593496]),
 'test_accuracy': array([0.74675325, 0.67532468, 0.64935065, 0.60784314, 0.7124183 ]),
 'train_accuracy': array([0.88599349, 0.88436482, 0.89576547, 0.88780488, 0.89593496])}

In [23]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'n_neighbors': list(range(1, 20)), 'p': [1, 2, 3]}
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [24]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.8868024132730016
{'n_neighbors': 3, 'p': 1}


In [25]:
# Cautare hiperparametrii optimi -RandomSearch
randomized_search = RandomizedSearchCV(estimator = KNeighborsClassifier(), param_distributions=parameter_grid, random_state=0,  scoring='accuracy', cv=4 )
randomized_search.fit(X_train, y1_train)
results_rscv = cross_val_score(randomized_search, X_test, y1_test, cv=5)

In [26]:
print(results_rscv.mean())
print(randomized_search.best_params_)

0.8204374057315234
{'p': 1, 'n_neighbors': 10}


## Model 3: DecisionTreeClassifier

In [27]:
# Model
"""
"""
model3 = DecisionTreeClassifier(max_depth=2)
model3.fit(X_train, y_train)
y_hat = model3.predict(X_test)

In [28]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[2. 2.]
 [1. 0.]
 [1. 0.]
 [2. 2.]
 [2. 2.]
 [0. 0.]
 [2. 2.]
 [2. 2.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[2. 1.]
 [1. 0.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [0. 0.]
 [2. 1.]
 [2. 2.]
 [1. 0.]
 [1. 0.]]

Failed:	[45 72]


In [29]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [30]:
# Cross Validation Y1
results_train = cross_validate(model3, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.0019927 , 0.00103092, 0.00099707, 0.0009973 , 0.00299311]),
 'score_time': array([0.00199676, 0.00196147, 0.00199556, 0.00199533, 0.00099611]),
 'test_f1_scorer': array([-0.79220779, -0.87012987, -0.87662338, -0.74509804, -0.79084967]),
 'train_f1_scorer': array([-0.82084691, -0.80130293, -0.79967427, -0.83252033, -0.82113821]),
 'test_accuracy': array([0.79220779, 0.87012987, 0.87662338, 0.74509804, 0.79084967]),
 'train_accuracy': array([0.82084691, 0.80130293, 0.79967427, 0.83252033, 0.82113821])}

In [31]:
# Cross Validation Y2
results_train = cross_validate(model3, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.0009985 , 0.00199485, 0.00099874, 0.00099659, 0.        ]),
 'score_time': array([0.00299072, 0.00199342, 0.00199485, 0.00199437, 0.00199795]),
 'test_f1_scorer': array([-0.66233766, -0.73376623, -0.73376623, -0.66666667, -0.62091503]),
 'train_f1_scorer': array([-0.79153094, -0.73452769, -0.73452769, -0.74471545, -0.75609756]),
 'test_accuracy': array([0.66233766, 0.73376623, 0.73376623, 0.66666667, 0.62091503]),
 'train_accuracy': array([0.79153094, 0.73452769, 0.73452769, 0.74471545, 0.75609756])}

In [32]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'max_depth': list(range(1,5)),
'min_samples_split': np.linspace(1,3,5),
'min_samples_leaf': np.linspace(0.1,0.5,5),
'max_features': ['sqrt', 'log2', None],
'criterion': ['gini', 'entropy']}
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [33]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.6797134238310709
{'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'min_samples_leaf': 0.4, 'min_samples_split': 1.0}


In [34]:
# Cautare hiperparametrii optimi -RandomSearch
randomized_search = RandomizedSearchCV(estimator = DecisionTreeClassifier(), param_distributions=parameter_grid, random_state=0,  scoring='accuracy', cv=4 )
randomized_search.fit(X_train, y1_train)
results_rscv = cross_val_score(randomized_search, X_test, y1_test, cv=5)

In [35]:
print(results_rscv.mean())
print(randomized_search.best_params_)

0.7150075414781296
{'min_samples_split': 1.0, 'min_samples_leaf': 0.2, 'max_features': None, 'max_depth': 3, 'criterion': 'gini'}


## Model 4: RandomForestClassifier

In [36]:
# Model
"""
"""
model4 = RandomForestClassifier(max_depth=2, random_state=0, ccp_alpha=0.2)
model4.fit(X_train, y_train)
y_hat = model4.predict(X_test)

In [37]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[3. 2.]
 [1. 0.]
 [1. 0.]
 [3. 2.]
 [3. 2.]
 [1. 0.]
 [3. 2.]
 [3. 2.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[2. 1.]
 [1. 0.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [0. 0.]
 [2. 1.]
 [2. 2.]
 [1. 0.]
 [1. 0.]]

Failed:	[80 72]


In [38]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [39]:
# Cross Validation Y1
results_train = cross_validate(model4, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.29989219, 0.25435328, 0.15359092, 0.16552567, 0.15857625]),
 'score_time': array([0.01196861, 0.01495409, 0.01093507, 0.00899267, 0.00797868]),
 'test_f1_scorer': array([-0.62987013, -0.70779221, -0.70779221, -0.7124183 , -0.7124183 ]),
 'train_f1_scorer': array([-0.71009772, -0.69055375, -0.69055375, -0.68943089, -0.68943089]),
 'test_accuracy': array([0.62987013, 0.70779221, 0.70779221, 0.7124183 , 0.7124183 ]),
 'train_accuracy': array([0.71009772, 0.69055375, 0.69055375, 0.68943089, 0.68943089])}

In [40]:
# Cross Validation Y2
results_train = cross_validate(model4, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.20245504, 0.15459132, 0.28426695, 0.20012569, 0.20015192]),
 'score_time': array([0.01196909, 0.01495695, 0.01694345, 0.00998044, 0.01695538]),
 'test_f1_scorer': array([-0.73376623, -0.73376623, -0.73376623, -0.73856209, -0.73202614]),
 'train_f1_scorer': array([-0.73452769, -0.73452769, -0.73452769, -0.73333333, -0.73495935]),
 'test_accuracy': array([0.73376623, 0.73376623, 0.73376623, 0.73856209, 0.73202614]),
 'train_accuracy': array([0.73452769, 0.73452769, 0.73452769, 0.73333333, 0.73495935])}

In [41]:
# Cautare hiperparametrii optimi - GridSearch (doar pentru Y1)
parameter_grid = {'max_depth': list(range(5,10)),
'min_samples_split': list(range(1,4)),
'min_samples_leaf': list(range(1,5))}
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [42]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.9647812971342382
{'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [43]:
# Cautare hiperparametrii optimi -RandomSearch
randomized_search = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions=parameter_grid, random_state=0,  scoring='accuracy', cv=4 )
randomized_search.fit(X_train, y1_train)
results_rscv = cross_val_score(randomized_search, X_test, y1_test, cv=5)

In [44]:
print(results_rscv.mean())
print(randomized_search.best_params_)

0.9570135746606334
{'min_samples_split': 3, 'min_samples_leaf': 1, 'max_depth': 7}


## Model 5: SVC (Support Vector Classification)

In [45]:
# Model
"""
Folosim metoda SVC pentru a clasifica observatiile. Atunci cand datele sunt in 4 sau mai multe dimensiuni SVC-ul este un hiper-plan. Acesta poate lucra cu valori extreme si din cazua ca permite clasificari eronate poate lucra si cu clasificari suprapuse.
"""
model5 = MultiOutputClassifier(SVC(C=2, gamma='auto'))
model5.fit(X_train, y_train)
y_hat = model5.predict(X_test)

In [46]:
print(f'Predicted:\n{y_hat[:10,:]}\n')
print(f'Ground truth:\n{y_test[:10,:]}\n')
print(f'Failed:\t{sum(y_hat != y_test)}')

Predicted:
[[2. 1.]
 [1. 0.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [1. 0.]
 [1. 0.]]

Ground truth:
[[2. 1.]
 [1. 0.]
 [1. 0.]
 [2. 1.]
 [3. 2.]
 [0. 0.]
 [2. 1.]
 [2. 2.]
 [1. 0.]
 [1. 0.]]

Failed:	[38 61]


In [47]:
y1_train = y_train[:, 0].reshape((y_train.shape[0], 1))
y2_train = y_train[:, 1].reshape((y_train.shape[0], 1))

y1_test = y_test[:, 0].reshape((y_test.shape[0], 1))
y2_test = y_test[:, 1].reshape((y_test.shape[0], 1))

In [48]:
# Cross Validation Y1
results_train = cross_validate(model5, X, y1,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.0009985 , 0.00099635, 0.00100064, 0.00099444, 0.00099826]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_f1_scorer': array([nan, nan, nan, nan, nan]),
 'train_f1_scorer': array([nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan])}

In [49]:
# Cross Validation Y2
results_train = cross_validate(model5, X, y2,
scoring={'f1_scorer': f1_scorer, 'accuracy': accuracy_scorer},
return_train_score=True)
results_train

{'fit_time': array([0.00099754, 0.00099611, 0.00099754, 0.        , 0.00096035]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_f1_scorer': array([nan, nan, nan, nan, nan]),
 'train_f1_scorer': array([nan, nan, nan, nan, nan]),
 'test_accuracy': array([nan, nan, nan, nan, nan]),
 'train_accuracy': array([nan, nan, nan, nan, nan])}

In [50]:
# Cautare hiperparametrii optimi - GridSearch
parameter_grid = {'C': np.linspace(0,1,5),
'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
'gamma': ['scale', 'auto']}
grid_search = GridSearchCV(estimator = SVC(), param_grid=parameter_grid, scoring='accuracy', cv=4, return_train_score=True)
grid_search.fit(X_train, y1_train)
results_gscv = cross_val_score(grid_search, X_test, y1_test, scoring='accuracy')

In [51]:
print(results_gscv.mean())
print(grid_search.best_params_)

0.8555806938159879
{'C': 0.75, 'gamma': 'scale', 'kernel': 'poly'}


In [52]:
# Cautare hiperparametrii optimi -RandomSearch
randomized_search = RandomizedSearchCV(estimator = SVC(), param_distributions=parameter_grid, random_state=0,  scoring='accuracy', cv=4 )
randomized_search.fit(X_train, y1_train)
results_rscv = cross_val_score(randomized_search, X_test, y1_test, cv=5)

In [53]:
print(results_rscv.mean())
print(randomized_search.best_params_)

0.8595022624434389
{'kernel': 'poly', 'gamma': 'scale', 'C': 0.75}
